In [1]:
import numpy as np
from shape_model.gaussian_ellipsoid import gaussian_ellipsoid_shape_model
from analytical_scattering_theories.homogeneous_sphere import mie_compute_q_and_s
from bl_dda.scatterer import Target, IncidentField, DiscreteDipoles



In [2]:
# Define the ellipsoid parameters a >= b >= c
#### length_unit= um ###
r_v_base= 0.2 # allowed range = [0.1, 0.5], volume equivalent radius of base ellipsoid
bc_ratio= 1 # allowed range = [1.0, 5], b/c ratio of base ellipsoid
ab_ratio= 1 # allowed range = [1.0, 2.0], a/b ratio of base ellipsoid
beta= 0.0 # standard deviation of GE surface deformation, allowed range= [0, 0.3]

In [3]:
gre_shape_model= gaussian_ellipsoid_shape_model(r_v_base, bc_ratio, ab_ratio, beta)

In [4]:
rng= np.random.default_rng(1234)
r_points_on_GRE_surf, xyz_meshes_GRE_surf = gre_shape_model.compute_r_points_on_GRE(rng)
lattice_domain, lattice_n, lattice_grid_points= gre_shape_model.create_cuboid_lattice_that_encloses_GRE_shape(r_points_on_GRE_surf)
dist_from_GRE= gre_shape_model.find_nearest_distance_from_the_GRE_surf(lattice_grid_points, r_points_on_GRE_surf)
lattice_grid_points_is_in_target= gre_shape_model.extract_lattice_address_in_GRE_volume(gre_shape_model.lattice_lf, gre_shape_model.distance_factor, lattice_n, dist_from_GRE)

In [ ]:
wl_0= 0.834
#m_m=1.329
m_m=1.0

'''
#### Refractive index data of minerals from mindat.org (@ mid-visible wavelength)
#Quartz Uniaxial, positive
m_p_x= 1.544+0j
m_p_y= 1.544+0j
m_p_z= 1.553+0j

#Mica(Muscovite) Biaxial
# m_p_x= 1.564+0j
# m_p_y= 1.599+0j
# m_p_z= 1.603+0j

#Kaolinite Biaxial
m_p_x= 1.558+0j
m_p_y= 1.564+0j
m_p_z= 1.565+0j

#Calcite Uniaxial, negative
m_p_x= 1.658+0j
m_p_y= 1.658+0j
m_p_z= 1.486+0j

#Aragonite Biaxial
m_p_x= 1.5295+0j
m_p_y= 1.681+0j
m_p_z= 1.6855+0j
'''

'''
#Bacteria
m_p_x= 1.36+0j
m_p_y= 1.36+0j
m_p_z= 1.36+0j

#Organics
m_p_x= 1.5+0j
m_p_y= 1.5+0j
m_p_z= 1.5+0j
'''

m_p_x= 1.58+0j
m_p_y= 1.58+0j
m_p_z= 1.58+0j


m_p_xyz= np.array([m_p_x, m_p_y, m_p_z], dtype=np.complex64)

m_p_avg= (m_p_x+m_p_y+m_p_z)/3
#m_p_xyz= np.array([m_p_avg, m_p_avg, m_p_avg], dtype=np.complex64)
m_p_avg

In [ ]:

target= Target(gre_shape_model.name, lattice_n, gre_shape_model.lattice_lf, lattice_grid_points, lattice_grid_points_is_in_target, m_p_xyz)
lattice_n
# np.prod(lattice_n)= 269705
lattice_n, np.prod(lattice_n)

In [ ]:
rng= np.random.default_rng()
number_of_orientations= 10
euler_alpha_deg= rng.uniform(0,360, number_of_orientations).reshape(number_of_orientations,1)
euler_beta_deg= rng.uniform(0,180, number_of_orientations).reshape(number_of_orientations,1)
euler_gamma_deg= rng.uniform(0,360, number_of_orientations).reshape(number_of_orientations,1)

# euler_alpha_deg= np.array([0,45+180], dtype=np.float32).reshape(number_of_orientations,1)
# euler_beta_deg= np.array([0,30], dtype=np.float32).reshape(number_of_orientations,1)
# euler_gamma_deg= np.array([0,0], dtype=np.float32).reshape(number_of_orientations,1)

euler_angles_radian= np.radians(np.hstack((euler_alpha_deg,euler_beta_deg,euler_gamma_deg)))

incident_field= IncidentField(wl_0, m_m, euler_angles_radian)

np.hstack((euler_alpha_deg,euler_beta_deg,euler_gamma_deg))

In [ ]:
discrete_dipoles= DiscreteDipoles(target, incident_field)

In [9]:
discrete_dipoles.set_interaction_matrix()

In [ ]:
discrete_dipoles.solve_matrix_equation()

#ray version nprocs=4, 42.7s 
#ray version nprocs=8, 34s 
#nonray version , 1m36.3s


In [ ]:
r_p= discrete_dipoles.ve_radius
r_p= r_v_base
m_p_avg= (m_p_x+m_p_y+m_p_z)/3
Q_sca_mie, Q_abs_mie, Q_ext_mie, S_fw_PCAS_mie, S_bk_OCBS_mie = mie_compute_q_and_s(wl_0,m_m,r_p,m_p_avg,nang=3)
Q_ext_mie, S_fw_PCAS_mie, S_bk_OCBS_mie

In [ ]:
C_abs= discrete_dipoles.compute_C_abs()
Q_abs= C_abs/(np.pi*discrete_dipoles.ve_radius**2)
Q_abs/Q_abs_mie, Q_abs, Q_abs_mie

In [ ]:
C_ext= discrete_dipoles.compute_C_ext()
Q_ext= C_ext/(np.pi*discrete_dipoles.ve_radius**2)
Q_ext/Q_ext_mie

In [ ]:
S_fw_PCAS_theta_P, S_fw_PCAS_phi_P= discrete_dipoles.compute_PCAS_observable_S_fw()
S_fw_PCAS_theta_P, S_fw_PCAS_mie

In [ ]:
S_fw_PCAS_phi_P, S_fw_PCAS_mie

In [16]:
S_bk_OCBS_P= discrete_dipoles.compute_OCBS_observable_S_bk()

In [ ]:
S_bk_OCBS_P, S_bk_OCBS_mie


In [18]:
ReS_PCAS_sp_avg, ImS_PCAS_sp_avg, ReS_PCAS_sp_avg_SNR, ImS_PCAS_sp_avg_SNR, ReS_PCAS_depol, ImS_PCAS_depol, ReS_PCAS_depol_SNR, ImS_PCAS_depol_SNR= discrete_dipoles.compute_PCAS_observable_S_fw_and_its_SNR()

In [ ]:
ReS_PCAS_sp_avg, ImS_PCAS_sp_avg

In [ ]:
ReS_PCAS_sp_avg_SNR, ImS_PCAS_sp_avg_SNR

In [ ]:
ReS_PCAS_depol, ImS_PCAS_depol

In [ ]:
ReS_PCAS_depol_SNR, ImS_PCAS_depol_SNR

# alpha=45, beta=30
# (array([9.7035768e-04, 9.7920007e-01], dtype=float32),
#  array([0.00339985, 2.5137208 ], dtype=float32))
